# Neptune + Fastai
## Before you start
**Install dependencies**

In [ ]:
! pip install --quiet neptune-client neptune-fastai fastai==2.6.3  torch==1.11.0 torchvision==0.12.0

## Basic Example

**Import libraries**

In [ ]:
import neptune.new as neptune
import torch
from fastai.callback.all import SaveModelCallback
from fastai.vision.all import (
    resnet18,
    vision_learner,
    ImageDataLoaders,
    untar_data,
    URLs,
    accuracy,
)
from neptune.new.integrations.fastai import NeptuneCallback
from neptune.new.types import File

### Create a Neptune run

To log metadata to the Neptune project, you need the `project name` and the `api_token`.

To make this example easy to follow, we have created a public project **'common/fastai-integration'** and a shared user **'neptuner'** with the API token **'ANONYMOUS'**. As you will see in the code cell below.

**(Optional)** To log to your Neptune project:

* [Create a Neptune account](https://app.neptune.ai/register/)

* [Find your API token](https://docs.neptune.ai/getting-started/installation#authentication-neptune-api-token)
* [Find your project name](https://docs.neptune.ai/getting-started/installation#setting-the-project-name)

Pass your credentials to project and api_token arguments of neptune.init()

`run = neptune.init(api_token='<YOUR_API_TOKEN>', project='<YOUR_WORKSPACE/YOUR_PROJECT>')` # pass your credentials


In [ ]:
run = neptune.init(
    project="common/fastai-integration",
    api_token="ANONYMOUS",
    tags="notebook-run",
)

**Dataset**

In [ ]:
path = untar_data(URLs.MNIST_TINY)
dls = ImageDataLoaders.from_csv(path)

In [ ]:
dls.show_batch()

### Log metadata using NeptuneCallback()
Using NeptuneCallback the following is automatically logged to Neptune UI for you:
- Hyperparameters
- Loss
- Metrics
- Best model weights 
- Model architecture


In [ ]:
learn = vision_learner(
    dls,
    resnet18,
    cbs=[SaveModelCallback(), NeptuneCallback(run=run, base_namespace="experiment")],
)

In [ ]:
learn.fit_one_cycle(1)

## More options

### Single and Multi phase logging

#### Log on a single training phase

In [ ]:
learn = vision_learner(dls, resnet18, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(1, cbs=[NeptuneCallback(run=run, base_namespace="experiment_1")])

#### Log all training phases of the learner

In [ ]:
learn = vision_learner(
    dls, resnet18, cbs=[NeptuneCallback(run=run, base_namespace="experiment_2")]
)

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.fit_one_cycle(2)

### Log model weights every n epochs

In [ ]:
n = 2
learn = vision_learner(
    dls,
    resnet18,
    metrics=accuracy,
    cbs=[
        SaveModelCallback(every_epoch=n),
        NeptuneCallback(
            run=run, base_namespace="experiment_3", upload_saved_models="all"
        ),
    ],
)

In [ ]:
learn.fit_one_cycle(5)

### Log images

In [ ]:
batch = dls.one_batch()
for i, (x, y) in enumerate(dls.decode_batch(batch)):
    # Neptune supports torch tensors
    # fastai uses their own tensor type name TensorImage
    # so you have to convert it back to torch.Tensor
    run["images/one_batch"].log(
        File.as_image(x.as_subclass(torch.Tensor).permute(2, 1, 0) / 255.0),
        name=f"{i}",
        description=f"Label: {y}",
    )

## Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()